In [136]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
browser = webdriver.Chrome()
url = r'https://www.nba.com/stats/teams/boxscores-advanced/?Season=2020-21&SeasonType=Regular%20Season'
browser.get(url)
select = Select(browser.find_element_by_xpath(r'/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'))
select.select_by_index(0)

src = browser.page_source
parser = BeautifulSoup(src, 'lxml')
table = parser.find('div', attrs = {'class': 'nba-stat-table__overflow'})
headers = table.findAll('th')
header_list = [h.text.strip() for h in headers[1:]]
header_list1 = [a for a in header_list if not 'RANK' in a]

In [3]:
header_list1

['Match\xa0Up',
 'Game\xa0Date',
 'Season',
 'W/L',
 'MIN',
 'OffRtg',
 'DefRtg',
 'NetRtg',
 'AST%',
 'AST/TO',
 'ASTRatio',
 'OREB%',
 'DREB%',
 'REB%',
 'TOV%',
 'eFG%',
 'TS%',
 'PACE',
 'PIE']

In [12]:
rows = table.findAll('tr')[1:]
box_scores = [[td.getText().strip() for td in rows[i].findAll('td')[1:]] for i in range(len(rows))]
adv_games = pd.DataFrame(box_scores, columns=header_list1)

In [13]:
adv_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Match Up   2160 non-null   object
 1   Game Date  2160 non-null   object
 2   Season     2160 non-null   object
 3   W/L        2160 non-null   object
 4   MIN        2160 non-null   object
 5   OffRtg     2160 non-null   object
 6   DefRtg     2160 non-null   object
 7   NetRtg     2160 non-null   object
 8   AST%       2160 non-null   object
 9   AST/TO     2160 non-null   object
 10  ASTRatio   2160 non-null   object
 11  OREB%      2160 non-null   object
 12  DREB%      2160 non-null   object
 13  REB%       2160 non-null   object
 14  TOV%       2160 non-null   object
 15  eFG%       2160 non-null   object
 16  TS%        2160 non-null   object
 17  PACE       2160 non-null   object
 18  PIE        2160 non-null   object
dtypes: object(19)
memory usage: 320.8+ KB


In [14]:
adv_games.head()

,Match Up,Game Date,Season,W/L,MIN,OffRtg,DefRtg,NetRtg,AST%,AST/TO,ASTRatio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE
0,SAS vs. PHX,05/16/2021,,L,48,117.5,118.3,-0.8,60.0,3.00,20.7,25.5,74.5,50.0,9.7,54.6,56.9,103.50,47.9
1,SAC vs. UTA,05/16/2021,,L,48,97.1,119.8,-22.7,57.1,1.85,18.0,14.3,74.5,43.8,12.7,51.7,52.4,101.50,40.3
2,CHA @ WAS,05/16/2021,,L,48,105.8,112.7,-7.0,67.6,1.92,17.4,25.4,72.9,46.7,12.5,46.8,52.6,103.00,47.5
3,CHI vs. MIL,05/16/2021,,W,48,113.5,107.7,5.8,51.1,1.44,16.8,25.0,74.1,52.0,15.4,59.8,61.9,104.00,55.4
4,IND @ TOR,05/16/2021,,W,48,117.9,107.6,10.3,70.8,2.62,22.4,27.1,67.8,49.5,12.3,58.4,60.7,105.50,57.2


In [15]:
browser = webdriver.Chrome()
url = r'https://www.nba.com/stats/teams/boxscores-traditional/?Season=2020-21&SeasonType=Regular%20Season'
browser.get(url)
select = Select(browser.find_element_by_xpath(r'/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'))
select.select_by_index(0)

src = browser.page_source
parser = BeautifulSoup(src, 'lxml')
table = parser.find('div', attrs = {'class': 'nba-stat-table__overflow'})
headers = table.findAll('th')
header_list = [h.text.strip() for h in headers[1:]]
header_list1 = [a for a in header_list if not 'RANK' in a]

In [18]:
rows = table.findAll('tr')[1:]
basic_box_scores = [[td.getText().strip() for td in rows[i].findAll('td')[1:]] for i in range(len(rows))]
basic_games = pd.DataFrame(basic_box_scores, columns=header_list1)

##### RERUN FROM HERE

In [194]:
all_boxes = basic_games.merge(adv_games, left_index=True, right_index=True)

all_boxes['Team'] = all_boxes['Match\xa0Up_x'].apply(lambda x: x[:3])
all_boxes['Opponent'] = all_boxes['Match\xa0Up_x'].apply(lambda x: x[-3:])
for i, val in enumerate(all_boxes['Match\xa0Up_x']):
    if val[4] == 'v':
        all_boxes['Match\xa0Up_x'][i] = 1
    if val[4] == '@':
        all_boxes['Match\xa0Up_x'][i] = 0

In [195]:
all_boxes = all_boxes.drop(columns=['Season_x', 'Season_y','W/L_y','MIN_y', 'Match\xa0Up_y', 'Game\xa0Date_y'])

In [196]:
all_boxes['Home'] = all_boxes['Match\xa0Up_x']
all_boxes['Date'] = all_boxes['Game\xa0Date_x']
all_boxes['Win'] = all_boxes['W/L_x']
all_boxes['Minutes'] = all_boxes['MIN_x']
all_boxes = all_boxes.drop(columns=['Match\xa0Up_x', 'Game\xa0Date_x', 'W/L_x', 'MIN_x'])

pd.set_option('display.max_columns', None)
all_boxes.head()

,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,+/-,OffRtg,DefRtg,NetRtg,AST%,AST/TO,ASTRatio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE,Team,Opponent,Home,Date,Win,Minutes
0,115,44,91,48.4,10,31,32.3,17,21,81.0,10,43,53,22,14,4,3,20,5,117.5,118.3,-0.8,60.0,3.00,20.7,25.5,74.5,50.0,9.7,54.6,56.9,103.50,47.9,WAS,CHA,1,05/16/2021,W,48
1,99,42,90,46.7,9,30,30.0,6,10,60.0,5,34,39,24,13,10,5,22,-22,97.1,119.8,-22.7,57.1,1.85,18.0,14.3,74.5,43.8,12.7,51.7,52.4,101.50,40.3,SAC,UTA,1,05/16/2021,L,48
2,117,50,94,53.2,8,26,30.8,9,16,56.3,14,40,54,20,15,1,12,11,5,105.8,112.7,-7.0,67.6,1.92,17.4,25.4,72.9,46.7,12.5,46.8,52.6,103.00,47.5,OKC,LAC,1,05/16/2021,W,48
3,96,34,79,43.0,13,27,48.1,15,19,78.9,4,37,41,16,11,7,6,17,4,113.5,107.7,5.8,51.1,1.44,16.8,25.0,74.1,52.0,15.4,59.8,61.9,104.00,55.4,NYK,BOS,1,05/16/2021,W,48
4,123,53,104,51.0,13,35,37.1,4,4,100,10,38,48,36,8,7,5,13,2,117.9,107.6,10.3,70.8,2.62,22.4,27.1,67.8,49.5,12.3,58.4,60.7,105.50,57.2,PHX,SAS,0,05/16/2021,W,48


In [198]:
# I need to find a way to changes the columns to floats.  

for col in all_boxes.drop(columns=['Team', 'Opponent', 'Home', 'Date', 'Win', 'Home']):
    all_boxes[col] = all_boxes[col].apply(lambda x: float(x))
            



In [205]:
all_boxes.to_csv('advanced_team_games.csv')

In [214]:
all_boxes['Date'] = pd.to_datetime(all_boxes['Date'])

In [215]:
all_boxes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   PTS       2160 non-null   float64       
 1   FGM       2160 non-null   float64       
 2   FGA       2160 non-null   float64       
 3   FG%       2160 non-null   float64       
 4   3PM       2160 non-null   float64       
 5   3PA       2160 non-null   float64       
 6   3P%       2160 non-null   float64       
 7   FTM       2160 non-null   float64       
 8   FTA       2160 non-null   float64       
 9   FT%       2160 non-null   float64       
 10  OREB      2160 non-null   float64       
 11  DREB      2160 non-null   float64       
 12  REB       2160 non-null   float64       
 13  AST       2160 non-null   float64       
 14  TOV       2160 non-null   float64       
 15  STL       2160 non-null   float64       
 16  BLK       2160 non-null   float64       
 17  PF        2160

In [212]:
all_boxes.groupby('Team').describe()

PTS                                                              FGM  \
     count        mean        std   min     25%    50%     75%    max count   
Team                                                                          
ATL   72.0  113.694444  11.889916  83.0  108.00  114.5  123.00  141.0  72.0   
BKN   72.0  118.569444  12.147945  88.0  110.75  117.0  127.25  147.0  72.0   
BOS   72.0  112.625000  12.932585  75.0  105.75  112.0  120.25  145.0  72.0   
CHA   72.0  109.458333  10.434150  86.0  103.50  110.0  114.25  135.0  72.0   
CHI   72.0  110.680556  11.360701  90.0  102.75  110.0  120.00  133.0  72.0   
CLE   72.0  103.833333  12.722388  75.0   96.00  103.0  111.00  147.0  72.0   
DAL   72.0  112.444444  11.512258  87.0  104.00  113.0  121.25  143.0  72.0   
DEN   72.0  115.055556  10.658448  87.0  108.75  115.0  122.00  139.0  72.0   
DET   72.0  106.611111  11.177890  81.0   99.00  107.0  115.00  132.0  72.0   
GSW   72.0  113.708333  12.983671  77.0  104.00  114.0  120.25  147.0  72.0   
HOU   72.0  108.791667  13.138415  84.0  100.00  107.0  116.25  143.0  72.0   
IND   72.0  115.305556  13.132734  87.0  107.75  113.5  122.50  152.0  72.0   
LAC   72.0  114.013889  12.375001  73.0  105.00  115.0  124.00  138.0  72.0   
LAL   72.0  109.541667  11.924572  86.0  101.00  109.5  117.00  138.0  72.0   
MEM   72.0  113.291667  12.493872  89.0  104.75  113.0  122.50  139.0  72.0   
MIA   72.0  108.069444  12.098570  80.0  101.75  108.5  116.00  134.0  72.0   
MIL   72.0  120.125000  11.778397  96.0  112.00  121.0  128.25  144.0  72.0   
MIN   72.0  112.125000  12.120216  81.0  103.75  111.0  120.25  137.0  72.0   
NOP   72.0  114.597222  11.298336  86.0  108.00  113.0  122.25  144.0  72.0   
NYK   72.0  107.027778  12.596637  83.0   98.50  106.0  114.25  140.0  72.0   
OKC   72.0  105.000000  10.941586  78.0   97.00  105.0  112.25  129.0  72.0   
ORL   72.0  104.000000  11.496478  75.0   96.00  103.0  112.00  130.0  72.0   
PHI   72.0  113.638889  11.623562  94.0  105.75  113.0  122.00  141.0  72.0   
PHX   72.0  115.277778  11.190064  85.0  107.00  116.0  123.00  140.0  72.0   
POR   72.0  116.111111  11.933567  87.0  106.00  116.0  125.00  141.0  72.0   
SAC   72.0  113.736111  10.029524  94.0  106.00  113.0  121.25  141.0  72.0   
SAS   72.0  111.083333  12.446788  85.0  102.75  111.0  119.00  146.0  72.0   
TOR   72.0  111.263889  11.198583  86.0  103.00  111.0  116.25  144.0  72.0   
UTA   72.0  116.416667  10.894824  95.0  110.75  116.0  122.25  154.0  72.0   
WAS   72.0  116.638889  13.697235  87.0  106.00  118.0  125.50  154.0  72.0   

                                                            FGA             \
           mean       std   min    25%   50%    75%   max count       mean   
Team                                                                         
ATL   40.791667  4.858382  27.0  37.75  40.0  44.00  53.0  72.0  87.236111   
BKN   43.138889  5.088804  32.0  40.00  43.0  47.00  57.0  72.0  87.347222   
BOS   41.458333  5.068371  25.0  39.00  41.0  45.00  52.0  72.0  88.902778   
CHA   39.930556  4.166690  30.0  37.00  40.0  43.00  51.0  72.0  87.833333   
CHI   42.152778  4.436277  33.0  39.00  42.5  45.00  55.0  72.0  88.611111   
CLE   38.583333  4.938352  29.0  36.00  38.5  41.00  56.0  72.0  85.763889   
DAL   41.083333  4.775967  31.0  38.00  41.0  44.00  51.0  72.0  87.319444   
DEN   43.250000  4.728547  32.0  40.00  43.0  46.00  56.0  72.0  89.194444   
DET   38.652778  4.797769  28.0  36.00  38.0  42.25  49.0  72.0  85.583333   
GSW   41.291667  5.276676  26.0  38.00  41.0  46.00  52.0  72.0  88.152778   
HOU   39.250000  5.327976  23.0  36.00  39.0  42.00  51.0  72.0  88.500000   
IND   43.263889  5.548742  34.0  38.75  43.5  46.00  57.0  72.0  91.208333   
LAC   41.791667  5.009674  26.0  38.00  42.0  45.00  54.0  72.0  86.694444   
LAL   40.625000  5.290005  32.0  36.00  41.0  44.00  52.0  72.0  86.069444   
MEM   42.847222  5.605982  30.0  38.75  42.0  47.00  55.0  72.0  91.77777